# Required libraries

In [ ]:
!huggingface-cli login

In [ ]:
!pip install -q transformers accelerate bitsandbytes gradio

In [ ]:
!pip install peft

In [ ]:
!pip install langchain faiss-cpu sentence-transformers

In [ ]:
!pip install -U langchain-community

#Rag Database

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vectorstore = FAISS.load_local(
    "/content/drive/MyDrive/rag_index",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)


# Data structure and Memory System

In [ ]:
import torch
import json
import os
from datetime import datetime # Import datetime
from typing import Dict, List
import gradio as gr

class SimpleDataManager:
    """Simple file-based storage for user data and recommendations"""

    def __init__(self, data_dir="wellbeing_data"):
        self.data_dir = data_dir
        os.makedirs(data_dir, exist_ok=True)
        os.makedirs(f"{data_dir}/users", exist_ok=True)
        os.makedirs(f"{self.data_dir}/recommendations", exist_ok=True) # Corrected path

    def save_user_week(self, user_id: str, week_data: Dict):
        """Save weekly data for a user"""
        user_file = f"{self.data_dir}/users/{user_id}.json"

        try:
            # Load existing data
            if os.path.exists(user_file):
                with open(user_file, 'r') as f:
                    user_data = json.load(f)
            else:
                user_data = {"user_id": user_id, "weeks": []}

            # Add new week
            user_data["weeks"].append(week_data)

            # Keep only last 8 weeks
            user_data["weeks"] = user_data["weeks"][-8:]

            # Save
            with open(user_file, 'w') as f:
                json.dump(user_data, f, indent=2)

            print(f"✅ Saved week data for user {user_id}")

        except Exception as e:
            print(f"❌ Error saving user data: {str(e)}")

    def get_user_history(self, user_id: str) -> List[Dict]:
        """Get user's weekly history"""
        user_file = f"{self.data_dir}/users/{user_id}.json"

        try:
            if os.path.exists(user_file):
                with open(user_file, 'r') as f:
                    user_data = json.load(f)
                    history = user_data.get("weeks", [])
                    print(f"📊 Retrieved {len(history)} weeks of history for {user_id}")
                    return history
        except Exception as e:
            print(f"❌ Error loading user history: {str(e)}")

        return []

    def save_recommendation(self, user_id: str, week_start: str, recommendation: str):
        """Save LLM recommendation"""
        # Clean week_start for filename (remove invalid characters)
        clean_week = week_start.replace("/", "-").replace(":", "-")
        rec_file = f"{self.data_dir}/recommendations/{user_id}_{clean_week}.json"

        try:
            rec_data = {
                "user_id": user_id,
                "week_start": week_start,
                "recommendation": recommendation,
                "timestamp": datetime.now().isoformat()
            }

            with open(rec_file, 'w') as f:
                json.dump(rec_data, f, indent=2)

            print(f"✅ Saved recommendation for user {user_id}, week {week_start}")

        except Exception as e:
            print(f"❌ Error saving recommendation: {str(e)}")

    def get_last_recommendation(self, user_id: str) -> str:
        """Get user's last recommendation"""
        rec_dir = f"{self.data_dir}/recommendations"

        try:
            # Check if recommendations directory exists
            if not os.path.exists(rec_dir):
                return ""

            # Find latest recommendation file for this user
            user_files = [f for f in os.listdir(rec_dir) if f.startswith(f"{user_id}_") and f.endswith('.json')]

            if user_files:
                # Sort by date and get latest
                user_files.sort(reverse=True)
                latest_file = f"{rec_dir}/{user_files[0]}"

                with open(latest_file, 'r') as f:
                    rec_data = json.load(f)
                    recommendation = rec_data.get("recommendation", "")
                    print(f"📝 Retrieved last recommendation for {user_id} ({len(recommendation)} chars)")
                    return recommendation

        except Exception as e:
            print(f"❌ Error loading last recommendation: {str(e)}")

        return ""

    def get_user_stats(self, user_id: str) -> Dict:
        """Get basic stats about user's data (bonus method)"""
        history = self.get_user_history(user_id)

        if not history:
            return {"total_weeks": 0}

        return {
            "total_weeks": len(history),
            "first_week": history[0].get("week_start", "Unknown"),
            "latest_week": history[-1].get("week_start", "Unknown"),
            "avg_steps": sum(week.get("total_steps", 0) for week in history) // len(history),
            "avg_sleep": sum(week.get("avg_sleep", 0) for week in history) / len(history)
        }

#Well being LLM

In [ ]:
import torch
import re
from typing import Dict, List


class WellbeingLLM:
    """Improved LLM system for wellbeing recommendations with better prompt engineering"""

    def __init__(self, base_model_id: str, adapter_path: str, vectorstore_path: str = None):
        self.model = None
        self.tokenizer = None
        self.device = None
        self.vectorstore = None

        # Load vectorstore if path provided
        if vectorstore_path:
            self.load_vectorstore(vectorstore_path)

        self.load_model(base_model_id, adapter_path)

    def load_vectorstore(self, vectorstore_path: str):
        """Load FAISS vectorstore for RAG"""
        try:
            from langchain.vectorstores import FAISS
            from langchain.embeddings import HuggingFaceEmbeddings

            print("📚 Loading knowledge base...")
            embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

            self.vectorstore = FAISS.load_local(
                vectorstore_path,
                embeddings=embedding_model,
                allow_dangerous_deserialization=True
            )
            print("✅ Knowledge base loaded")

        except Exception as e:
            print(f"⚠️ Could not load vectorstore: {e}")
            self.vectorstore = None

    def load_model(self, base_model_id: str, adapter_path: str = None):
        """Load the LLM model with better error handling"""
        try:
            print("🤖 Loading LLM...")

            from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
            from peft import PeftModel

            # Clear GPU memory and check device
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
                print(f"🔍 GPU available: {torch.cuda.get_device_name()}")
            else:
                print("🔍 Using CPU")

            # Load tokenizer
            print("📝 Loading tokenizer...")
            self.tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token
            print("✅ Tokenizer loaded")

            # Quantization config
            bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_use_double_quant=True,
                bnb_4bit_compute_dtype=torch.bfloat16,
            )
            print("⚙️ Quantization config ready")

            # Load base model
            print("🧠 Loading base model...")
            self.model = AutoModelForCausalLM.from_pretrained(
                base_model_id,
                device_map="auto",
                torch_dtype=torch.float16,  # Faster than float16
                quantization_config=bnb_config,
                trust_remote_code=True,
                low_cpu_mem_usage=True,      # 🚀 NEW: Reduces CPU memory usage
                use_cache=False,             # 🚀 NEW: Faster loading
            )
            print("✅ Base model loaded")

            # Load fine-tuned adapter
            # print("🎯 Loading fine-tuned adapter...")
            # self.model = PeftModel.from_pretrained(self.model, adapter_path)
            self.model.eval()

            # Store device for later use
            self.device = next(self.model.parameters()).device
            print(f"✅ LLM loaded successfully on {self.device}")

        except Exception as e:
            print(f"❌ LLM loading failed: {str(e)}")
            import traceback
            traceback.print_exc()
            self.model = None
            self.tokenizer = None
            self.device = None






    def generate_recommendation(self, current_week: Dict, user_history: List[Dict], last_recommendation: str) -> str:
      """Generate personalized recommendations with single template enforcement"""

      if not self.model or not self.tokenizer:
          print("❌ No model loaded")
          return "Model not available. Please check model loading."

      try:



          # Use the single master template
          query = self._build_master_template(current_week, user_history)


          docs = self.vectorstore.similarity_search(query, k=3)


          context = "\n\n".join(doc.page_content for doc in docs)
          print(f"the given prompt is {query}")


          prompt = (
          f"You are a helpful medical assistant. Use the following context to answer the question.\n\n"
          f"Context:\n{context}\n\n"
          f"Question: {query}\n\nAnswer:"
      )
          inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
          input_len = inputs["input_ids"].shape[1]

          print(f"Model is on: {self.device}")  # Should be 'cuda'


          with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=500,
                temperature=0.1,
                top_p=0.95,
                do_sample=True
            )

          generated_tokens = outputs[0][input_len:]
          answer = self.tokenizer.decode(generated_tokens, skip_special_tokens=True)

          return answer.strip()

      except Exception as e:
          print(f"❌ Generation error: {e}")
          return "Failed to generate response."


    def _build_master_template(self, current_week: Dict, user_history: List[Dict]) -> str:
      """Single master template with RAG-enhanced knowledge"""

      # Extract user data
      total_steps = current_week.get('total_steps', 0)
      zone_minutes = current_week.get('zone_minutes', 0)
      demographics = current_week.get('demographics', {})
      age = demographics.get('age', 'unknown')
      weight = demographics.get('weight', 'unknown')
      height = demographics.get('height', 'unknown')
      sex = demographics.get('sex', 'unknown')
      avg_sleep = current_week.get('avg_sleep', 0)
      diet_type = current_week.get('preferences', {}).get('diet_type', 'No Preference')
      allergies = current_week.get('preferences', {}).get('allergies', [])
      exercise_sessions = current_week.get('exercise_sessions', 0)
      food_data = current_week.get('food_data', {})
      dairy_liters = food_data.get('dairy_liters',0)
      legumes = food_data.get('legumes_grams',0)
      meat = food_data.get('meat_grams',0)
      furits= food_data.get('fruits_grams',0)
      vegtables = food_data.get('vegetables_grams',0)
      grains = food_data.get('grains_grams',0)
      nuts = food_data.get('nuts_seeds_grams',0)
      water = food_data.get('water_liters',0)



      # The master template with RAG integration
      prompt = f"""
A user is a {sex} aged {age} years, with a height of {height} cm and weight of {weight} kg. They walk about {total_steps:,} steps weekly, average {zone_minutes} zone minutes, and do {exercise_sessions} exercise sessions per week. They sleep around {avg_sleep:.1f} hours each night. Their diet preference is {diet_type}, and they are allergic to {', '.join(allergies) if allergies else 'none'}
and their this weekly food consumption is dairy {dairy_liters} liter, legumes {legumes} grams, meat {meat} grams, fruits {furits} grams, vegtables = {vegtables},
 grains {grains} grams, nuts {nuts} grams and water {water} liter .
---
**Important Dietary Notes:**
- Provide food suggestions appropriate for the user's diet preference ({diet_type}).
- If the user consumes foods that contradict their diet or allergies, highlight these inconsistencies and offer recommendations to align their consumption with their dietary needs.
---
Based on this profile and the expert knowledge below, provide health and wellness recommendations in the format outlined.

---

**1) Food Recommendation**
- Overall Assessment: [...]
- Areas of Improvements: [...]
- Suggested Meals: [...]

**2) Physical Activity**
- Activity Assessment: [...]
- Zone Minutes and intensity Feedback
- Strength/Cardio Tips: [suggest also exercise for strength and cardio]
- Weekly Goals: [...]

**3) Sleep & Well-being**
- Sleep Review: [...]
- Suggestions: [...]

**4) Weekly Summary**
- Summary: [...]
- Goals: [...]
"""


      return prompt




# Well being system

In [ ]:
class WellbeingSystem:
    """Main system that combines data management and LLM"""

    def __init__(self, base_model_id: str, adapter_path: str, vectorstore_path: str = None):
        self.data_manager = SimpleDataManager()
        self.llm = WellbeingLLM(base_model_id, adapter_path, vectorstore_path) # Pass vectorstore_path to LLM
        print("🎯 WellbeingSystem initialized")

    def analyze_and_recommend(self, user_id: str, week_data: Dict) -> str:
        """Main function: analyze weekly data and generate recommendations"""

        print(f"📊 Analyzing data for user: {user_id}")

        try:
            # Get user history and last recommendation
            user_history = self.data_manager.get_user_history(user_id)
            last_recommendation = self.data_manager.get_last_recommendation(user_id)

            print(f"📈 Found {len(user_history)} weeks of history")
            if last_recommendation:
                print(f"📝 Previous recommendation found ({len(last_recommendation)} chars)")

            # Process current week data
            processed_week = self._process_week_data(week_data)

            # Show what data is being sent to LLM
            print(f"📊 Processed data: {processed_week['total_steps']:,} steps, {processed_week['zone_minutes']} zone mins, {processed_week['avg_sleep']:.1f}h sleep")

            # Generate recommendation using LLM
            recommendation = self.llm.generate_recommendation(
                processed_week, user_history, last_recommendation
            )

            # Save data (with error handling)
            try:
                self.data_manager.save_user_week(user_id, processed_week)
                self.data_manager.save_recommendation(user_id, processed_week['week_start'], recommendation)
                print(f"💾 Data saved successfully")
            except Exception as save_error:
                print(f"⚠️ Warning: Could not save data: {save_error}")
                # Continue anyway - return the recommendation even if saving fails

            print(f"✅ Generated recommendation for {user_id} ({len(recommendation)} chars)")
            return recommendation

        except Exception as e:
            print(f"❌ Error in analyze_and_recommend: {str(e)}")
            import traceback
            traceback.print_exc()
            return f"Analysis failed: {str(e)}. Please check your input data."

    def _process_week_data(self, week_data: Dict) -> Dict:
        """Process and standardize weekly data with better validation"""

        try:
            # Parse sleep and mood data with validation
            sleep_hours = week_data.get('sleep_hours', [])
            mood_scores = week_data.get('mood_scores', [])

            # Handle string input (comma-separated values)
            if isinstance(sleep_hours, str):
                try:
                    sleep_hours = [float(x.strip()) for x in sleep_hours.split(',') if x.strip()]
                except ValueError:
                    print("⚠️ Invalid sleep hours format, using default")
                    sleep_hours = []

            if isinstance(mood_scores, str):
                try:
                    mood_scores = [float(x.strip()) for x in mood_scores.split(',') if x.strip()]
                except ValueError:
                    print("⚠️ Invalid mood scores format, using default")
                    mood_scores = []

            # Calculate averages with validation
            avg_sleep = sum(sleep_hours) / len(sleep_hours) if sleep_hours else 0
            avg_mood = sum(mood_scores) / len(mood_scores) if mood_scores else 0

            # 🚀 ADD DEBUG:
            print(f"🔍 DEBUG - Sleep calculation: {sleep_hours} → avg: {avg_sleep}")
            print(f"🔍 DEBUG - Mood calculation: {mood_scores} → avg: {avg_mood}")

            # Enhanced food data processing
            food_data = self._process_food_data(week_data)

            processed_data = {
                'week_start': week_data.get('week_start', datetime.now().strftime('%Y-%m-%d')),
                'total_steps': max(0, int(week_data.get('total_steps', 0))), # Ensure int and non-negative
                'zone_minutes': max(0, int(week_data.get('zone_minutes', 0))), # Ensure int and non-negative
                'exercise_sessions': max(0, int(week_data.get('exercise_sessions', 0))), # Ensure int and non-negative
                'avg_sleep': round(avg_sleep, 1),
                'avg_mood': round(avg_mood, 1),
                'food_data': food_data,
                'timestamp': datetime.now().isoformat(),
                'demographics': week_data.get('demographics', {})
            }

            # Include preferences and targets if present in the input week_data
            if 'preferences' in week_data:
                processed_data['preferences'] = week_data['preferences']
            if 'targets' in week_data:
                processed_data['targets'] = week_data['targets']


            return processed_data

        except Exception as e:
            print(f"❌ Error processing week data: {str(e)}")
            # Return empty or default data structure on error
            return {
                 'week_start': week_data.get('week_start', datetime.now().strftime('%Y-%m-%d')),
                 'total_steps': 0,
                 'zone_minutes': 0,
                 'exercise_sessions': 0,
                 'avg_sleep': 0.0,
                 'avg_mood': 0.0,
                 'food_data': {},
                 'timestamp': datetime.now().isoformat(),
                 'processing_error': str(e)
            }


    def _process_food_data(self, week_data: Dict) -> Dict:
        """Process food data handling both old and new formats"""

        food_data = {}

        # Check for new format first (with units)
        unit_fields = ['dairy_liters', 'water_liters', 'legumes_grams', 'meat_grams',
                      'fruits_grams', 'vegetables_grams', 'grains_grams', 'nuts_seeds_grams']

        has_unit_format = any(field in week_data for field in unit_fields)

        if has_unit_format:
            print("📊 Using new format (with units)")
            # New format with units
            for field in unit_fields:
                value = week_data.get(field, 0)
                try:
                    food_data[field] = max(0, float(value))  # Ensure non-negative
                except (ValueError, TypeError):
                    food_data[field] = 0
                    print(f"⚠️ Invalid {field} value, using 0")
        else:
            print("📊 Using old format (servings)")
            # Old format (for backward compatibility)
            old_fields = ['dairy', 'legumes', 'meat', 'fruits', 'vegetables', 'grains', 'nuts_seeds', 'water_glasses']
            for field in old_fields:
                if field in week_data:
                    try:
                        food_data[field] = max(0, float(week_data[field]))
                    except (ValueError, TypeError):
                        food_data[field] = 0

        return food_data





# Well being app

In [ ]:
import gradio as gr
import json
import os
from datetime import datetime
from typing import Dict, List

class WellbeingApp:
    """Complete Gradio interface with targets, history, and preferences"""

    def __init__(self, base_model_id: str, adapter_path: str):
      print("🚀 Initializing Complete WellbeingApp...")
      self.wellbeing_system = WellbeingSystem(
          base_model_id,
          adapter_path,
          vectorstore_path="/content/drive/MyDrive/rag_index"
      )

      # Default targets (same as before)
      self.default_targets = {
          'weekly_steps': 70000,
          'weekly_zone_minutes': 150,
          'daily_sleep_hours': 8.0,
          'weekly_water_liters': 14.0,
          'weekly_fruits_grams': 2100,
          'weekly_vegetables_grams': 3500,
          'weekly_exercise_sessions': 4
      }

    def create_wellbeing_app(self):
        """Create comprehensive Gradio interface with tabs"""

        with gr.Blocks(title="🏥 Complete Wellbeing LLM", theme=gr.themes.Soft()) as demo:

            gr.Markdown("# 🏥 Complete AI Wellbeing System")
            gr.Markdown("### Personalized health recommendations with custom targets and progress tracking")

            # Create tabs for different sections
            with gr.Tabs():

                # TAB 1: MAIN ANALYSIS
                with gr.TabItem("🤖 Get Recommendations", elem_id="main-tab"):
                    self._create_main_analysis_tab()

                # TAB 2: TARGETS CUSTOMIZATION
                with gr.TabItem("🎯 Customize Targets", elem_id="targets-tab"):
                    self._create_targets_tab()

                # TAB 3: HISTORY REVIEW
                with gr.TabItem("📈 Progress History", elem_id="history-tab"):
                    self._create_history_tab()

                # TAB 4: SYSTEM INFO
                # with gr.TabItem("ℹ️ System Info", elem_id="info-tab"):
                #     self._create_info_tab()

        return demo

    def _create_main_analysis_tab(self):
      """Create the main analysis tab with fixed component handling"""

      with gr.Row():
          with gr.Column():

              gr.Markdown("### 🤖 AI Enhancement Options")
              # User Info Section
              gr.Markdown("### 👤 User Information")
              user_id = gr.Textbox(label="👤 User ID", value="user_001", info="Unique identifier for tracking your progress")
              week_start = gr.Textbox(label="📅 Week Start Date", value="2024-01-15", info="Format: YYYY-MM-DD")

              # Personal Details - FIXED: These were not being collected properly
              sex = gr.Dropdown(
                  label="⚧️ Sex",
                  choices=["Male", "Female", "Other", "Prefer not to say"],
                  value="Male",
                  info="Biological sex for health calculations"
              )

              age = gr.Number(
                  label="🎂 Age",
                  value=25,
                  minimum=13,
                  maximum=120,
                  step=1,
                  info="Age in years"
              )

              weight = gr.Number(
                  label="⚖️ Weight (kg)",
                  value=70.0,
                  minimum=30.0,
                  maximum=300.0,
                  step=0.1,
                  info="Current weight in kilograms"
              )

              height = gr.Number(
                  label="📏 Height (cm)",
                  value=175.0,
                  minimum=100.0,
                  maximum=250.0,
                  step=0.1,
                  info="Height in centimeters"
              )

              # Preferences Section
              gr.Markdown("### 🍽️ Meal Preferences & Dietary Restrictions")
              with gr.Row():
                  diet_type = gr.Dropdown(
                      label="🥗 Diet Type",
                      choices=[
                          "No Preference", "Vegetarian", "Vegan", "Pescatarian",
                          "Keto", "Paleo", "Mediterranean", "Low Carb", "Gluten Free", "Dairy Free"
                      ],
                      value="No Preference",
                      info="Select your primary dietary preference"
                  )

              allergies = gr.CheckboxGroup(
                  label="⚠️ Food Allergies & Intolerances",
                  choices=[
                      "Nuts (Tree nuts)", "Peanuts", "Shellfish", "Fish",
                      "Milk/Dairy", "Eggs", "Soy", "Wheat/Gluten",
                      "Sesame", "Lactose Intolerant", "Other"
                  ],
                  info="⚠️ IMPORTANT: Select all allergies and intolerances"
              )

              other_allergies = gr.Textbox(
                  label="🚨 Other Allergies/Restrictions",
                  placeholder="e.g., tomatoes, specific medications, religious restrictions...",
                  info="Specify any other dietary restrictions or allergies"
              )

              # Activity Section
              gr.Markdown("### 🏃‍♂️ Physical Activity")
              total_steps = gr.Number(label="Total Steps (Week)", value=65000, info="Your current target will be shown")
              zone_minutes = gr.Number(label="Zone Minutes (Week)", value=120, info="Your current target will be shown")
              exercise_sessions = gr.Number(label="Exercise Sessions", value=3, info="Your current target will be shown")

              # Sleep & Mood Section
              gr.Markdown("### 😴 Sleep & Mood")
              sleep_hours = gr.Textbox(
                  label="Sleep Hours (7 days)",
                  value="7.5,6.8,8.2,7.0,6.5,8.5,7.8",
                  info="Enter daily sleep hours separated by commas"
              )
              mood_scores = gr.Textbox(
                  label="Mood Scores (7 days)",
                  value="4,3.5,5,3,4,2.5,5",
                  info="Rate your daily mood from 1-5, separated by commas"
              )

              # Food Section
              gr.Markdown("### 🍎 Weekly Nutrition Intake")
              gr.Markdown("*Enter your total consumption for the entire week*")

              with gr.Row():
                  dairy = gr.Number(label="🥛 Dairy (liters)", value=3.5, info="Milk, yogurt, cheese")
                  water_liters = gr.Number(label="💧 Water (liters)", value=48, info="Your current target will be shown")

              with gr.Row():
                  fruits = gr.Number(label="🍎 Fruits (grams)", value=2100, info="Your current target will be shown")
                  vegetables = gr.Number(label="🥬 Vegetables (grams)", value=3500, info="Your current target will be shown")

              with gr.Row():
                  legumes = gr.Number(label="🫘 Legumes (grams)", value=350, info="Beans, lentils, peas")
                  nuts_seeds = gr.Number(label="🥜 Nuts/Seeds (grams)", value=210, info="Almonds, walnuts, chia seeds")

              with gr.Row():
                  meat = gr.Number(label="🍗 Meat (grams)", value=700, info="Chicken, beef, fish")
                  grains = gr.Number(label="🌾 Grains (grams)", value=2800, info="Rice, bread, pasta, oats")

              # Action buttons
              with gr.Row():
                  analyze_btn = gr.Button("🤖 Get AI Recommendations", variant="primary", size="lg")
                  load_targets_btn = gr.Button("📊 Load My Targets", variant="secondary")

          with gr.Column():
              main_output = gr.Markdown("""
  Suggestions
  """)

      # FIXED: Store components with correct order including demographics
      self.main_components = {
          'user_id': user_id,
          'week_start': week_start,
          'sex': sex,           # ADDED: Missing demographic fields
          'age': age,           # ADDED
          'weight': weight,     # ADDED
          'height': height,     # ADDED
          'total_steps': total_steps,
          'zone_minutes': zone_minutes,
          'exercise_sessions': exercise_sessions,
          'sleep_hours': sleep_hours,
          'mood_scores': mood_scores,
          'dairy_liters': dairy,
          'legumes_grams': legumes,
          'meat_grams': meat,
          'fruits_grams': fruits,
          'vegetables_grams': vegetables,
          'grains_grams': grains,
          'nuts_seeds_grams': nuts_seeds,
          'water_liters': water_liters,
          'diet_type': diet_type,
          'allergies': allergies,
          'other_allergies': other_allergies,
          'output': main_output
      }

      # FIXED: Connect button with correct inputs order
      analyze_btn.click(
          fn=self.analyze_wellbeing_with_preferences,
          inputs=[
              user_id, week_start, sex, age, weight, height,  # Demographics in correct order
              total_steps, zone_minutes, exercise_sessions,
              sleep_hours, mood_scores, dairy, legumes, meat,
              fruits, vegetables, grains, nuts_seeds, water_liters,
              diet_type, allergies, other_allergies
          ],
          outputs=[main_output]
      )

      load_targets_btn.click(
          fn=self.load_user_targets,
          inputs=[user_id],
          outputs=[main_output]
      )

    def analyze_wellbeing_with_preferences(self, user_id, week_start, sex, age, weight, height,
                                     total_steps, zone_minutes, exercise_sessions,
                                     sleep_hours, mood_scores, dairy_liters, legumes_grams, meat_grams,
                                     fruits_grams, vegetables_grams, grains_grams, nuts_seeds_grams, water_liters,
                                     diet_type, allergies, other_allergies):
      """Enhanced analysis with agent support, custom targets, and user demographics"""

      try:
          # Input validation
          if not user_id or not user_id.strip():
              return "❌ Error: Please enter a valid User ID"

          if not week_start:
              return "❌ Error: Please enter a week start date"

          # Validate demographic data
          if age is not None and (age < 13 or age > 120):
              return "❌ Error: Please enter a valid age (13-120 years)"

          if weight is not None and (weight < 30 or weight > 300):
              return "❌ Error: Please enter a valid weight (30-300 kg)"

          if height is not None and (height < 100 or height > 250):
              return "❌ Error: Please enter a valid height (100-250 cm)"

          # Load user's custom targets
          user_targets = self.get_user_targets(user_id.strip())

          # Calculate BMI if weight and height are provided
          bmi = None
          if weight and height:
              height_m = height / 100  # Convert cm to meters
              bmi = round(weight / (height_m ** 2), 1)

          # Prepare enhanced data with demographics
          week_data = {
              'user_id': user_id.strip(),
              'week_start': week_start,
              'demographics': {
                  'sex': sex,
                  'age': int(age) if age else None,
                  'weight': float(weight) if weight else None,
                  'height': float(height) if height else None,
                  'bmi': bmi
              },
              'total_steps': max(0, int(total_steps)) if total_steps else 0,
              'zone_minutes': max(0, int(zone_minutes)) if zone_minutes else 0,
              'exercise_sessions': max(0, int(exercise_sessions)) if exercise_sessions else 0,
              'sleep_hours': sleep_hours,
              'mood_scores': mood_scores,
              'dairy_liters': max(0, float(dairy_liters)) if dairy_liters else 0,
              'legumes_grams': max(0, float(legumes_grams)) if legumes_grams else 0,
              'meat_grams': max(0, float(meat_grams)) if meat_grams else 0,
              'fruits_grams': max(0, float(fruits_grams)) if fruits_grams else 0,
              'vegetables_grams': max(0, float(vegetables_grams)) if vegetables_grams else 0,
              'grains_grams': max(0, float(grains_grams)) if grains_grams else 0,
              'nuts_seeds_grams': max(0, float(nuts_seeds_grams)) if nuts_seeds_grams else 0,
              'water_liters': max(0, float(water_liters)) if water_liters else 0,
              'preferences': {
                  'diet_type': diet_type,
                  'allergies': allergies if allergies else [],
                  'other_allergies': other_allergies.strip() if other_allergies else ""
              },
              'targets': user_targets
          }

          recommendation = self.wellbeing_system.analyze_and_recommend(user_id.strip(), week_data)

          # Enhanced output with custom targets and demographics
          output = self._format_analysis_output_with_targets(week_data, recommendation, user_targets)

          return output

      except Exception as e:
          print(f"❌ Analysis error: {str(e)}")
          import traceback
          traceback.print_exc()
          return f"❌ Error: {str(e)}\n\nPlease check your input format and try again."


    def _create_targets_tab(self):
        """Create the targets customization tab"""

        gr.Markdown("## 🎯 Customize Your Personal Health Targets")
        gr.Markdown("### Set personalized goals based on your fitness level, health conditions, and personal objectives")

        with gr.Row():
            with gr.Column():
                gr.Markdown("### 👤 User Selection")
                target_user_id = gr.Textbox(label="👤 User ID", value="user_001", info="Enter the user ID to customize targets for")

                gr.Markdown("### 🏃‍♂️ Activity Targets")
                target_weekly_steps = gr.Number(
                    label="Weekly Steps Target",
                    value=70000,
                    info="Default: 70,000 (10,000 daily). Adjust based on fitness level."
                )
                target_zone_minutes = gr.Number(
                    label="Weekly Zone Minutes Target",
                    value=150,
                    info="Default: 150 minutes. WHO recommends 150+ minutes moderate exercise."
                )
                target_exercise_sessions = gr.Number(
                    label="Weekly Exercise Sessions Target",
                    value=4,
                    info="Default: 4 sessions. Include both cardio and strength training."
                )

                gr.Markdown("### 😴 Sleep & Recovery Targets")
                target_daily_sleep = gr.Number(
                    label="Daily Sleep Hours Target",
                    value=8.0,
                    info="Default: 8 hours. Range: 7-9 hours for most adults."
                )

                with gr.Row():
                    save_targets_btn = gr.Button("💾 Save My Targets", variant="primary", size="lg")
                    load_current_targets_btn = gr.Button("📊 Load Current Targets", variant="secondary")
                    reset_defaults_btn = gr.Button("🔄 Reset to Defaults", variant="secondary")


        # Store target components
        self.target_components = {
            'user_id': target_user_id,
            'weekly_steps': target_weekly_steps,
            'zone_minutes': target_zone_minutes,
            'exercise_sessions': target_exercise_sessions,
            'daily_sleep': target_daily_sleep,
        }

        # Connect buttons
        save_targets_btn.click(
            fn=self.save_user_targets,
            inputs=list(self.target_components.values())[:-1],
            # outputs=[targets_output]
        )

        load_current_targets_btn.click(
            fn=self.load_user_targets_display,
            inputs=[target_user_id],
            outputs=list(self.target_components.values()) # Update all target number boxes and output
        )

        reset_defaults_btn.click(
            fn=self.reset_to_defaults,
            outputs=list(self.target_components.values())
        )


    def _create_history_tab(self):
        """Create the history review tab"""

        gr.Markdown("## 📈 Progress History & Analytics")
        gr.Markdown("### Track your health journey and see improvements over time")

        with gr.Row():
            with gr.Column():
                gr.Markdown("### 👤 User Selection")
                history_user_id = gr.Textbox(label="👤 User ID", value="user_001", info="Enter user ID to view history")

                gr.Markdown("### 📊 Analysis Options")
                analysis_type = gr.Radio(
                    label="📈 View Type",
                    choices=[
                        "📊 Complete Progress Summary",
                        "📈 Weekly Trends Analysis",
                        "🎯 Target Achievement Report",
                        "📋 Raw Data Export",
                        "🏆 Achievement Milestones"
                    ],
                    value="📊 Complete Progress Summary",
                    info="Choose what type of analysis you want to see"
                )

                weeks_to_show = gr.Slider(
                    label="📅 Weeks to Include",
                    minimum=1,
                    maximum=12,
                    value=4,
                    step=1,
                    info="How many recent weeks to analyze"
                )

                with gr.Row():
                    view_history_btn = gr.Button("📊 View Progress", variant="primary", size="lg")
                    export_data_btn = gr.Button("💾 Export Data", variant="secondary")
                    delete_history_btn = gr.Button("🗑️ Clear History", variant="stop")



        # Store history components
        self.history_components = {
            'user_id': history_user_id,
            'analysis_type': analysis_type,
            'weeks_to_show': weeks_to_show,
            # 'output': history_output
        }

    def _format_preferences_display(self, preferences: dict) -> str:
        """Format preferences for nice display"""

        lines = ["### 🍽️ Your Dietary Profile:"]

        # Diet type
        if preferences.get('diet_type', 'No Preference') != "No Preference":
            lines.append(f"- **Diet:** {preferences['diet_type']}")

        # Allergies (most important!)
        if preferences.get('allergies'):
            allergy_list = ", ".join(preferences['allergies'])
            lines.append(f"- **⚠️ ALLERGIES:** {allergy_list}")

        if preferences.get('other_allergies'):
            lines.append(f"- **⚠️ OTHER RESTRICTIONS:** {preferences['other_allergies']}")

        if len(lines) == 1:  # Only the header
            lines.append("- No specific dietary preferences set")

        return "\n".join(lines)



    def _format_analysis_output_with_targets(self, week_data, recommendation, user_targets):
      """Format analysis output including custom targets"""

      daily_steps = week_data['total_steps'] / 7
      daily_water = week_data['water_liters'] / 7

      # Safely get avg_sleep and avg_mood with defaults
      avg_sleep = week_data.get('avg_sleep', 0)
      avg_mood = week_data.get('avg_mood', 0)

      # Compare against custom targets
      targets_comparison = f"""
  ### 🎯 Target Performance:
  - **Steps:** {week_data['total_steps']:,} / {user_targets['weekly_steps']:,} {'✅' if week_data['total_steps'] >= user_targets['weekly_steps'] else '⚠️'} ({((week_data['total_steps'] / user_targets['weekly_steps']) * 100):.0f}%)
  - **Zone Minutes:** {week_data['zone_minutes']} / {user_targets['weekly_zone_minutes']} {'✅' if week_data['zone_minutes'] >= user_targets['weekly_zone_minutes'] else '⚠️'} ({((week_data['zone_minutes'] / user_targets['weekly_zone_minutes']) * 100):.0f}%)
  - **Sleep:** {avg_sleep:.1f}h / {user_targets['daily_sleep_hours']}h {'✅' if avg_sleep >= user_targets['daily_sleep_hours'] else '⚠️'}
  - **Water:** {week_data['water_liters']:.1f}L / {user_targets['weekly_water_liters']}L {'✅' if week_data['water_liters'] >= user_targets['weekly_water_liters'] else '⚠️'}
"""

      preferences_display = self._format_preferences_display(week_data['preferences'])

      return f"""# 🏥 Personalized Wellbeing Analysis & Recommendations

  ## 👤 User: {week_data['user_id']} | 📅 Week: {week_data['week_start']}

  {preferences_display}

  {targets_comparison}

  ### 📊 Your Weekly Data Summary:
  **🏃‍♂️ Activity:**
  - **Steps:** {week_data['total_steps']:,} ({daily_steps:.0f}/day)
  - **Zone Minutes:** {week_data['zone_minutes']}
  - **Exercise Sessions:** {week_data['exercise_sessions']}

  **😴 Sleep & Mood:**
  - **Sleep:** {week_data['sleep_hours']} hours daily
  - **Mood:** {week_data['mood_scores']}/10 daily scores

  **🍎 Nutrition:**
  - 🥛 **Dairy:** {week_data['dairy_liters']:.1f}L | 💧 **Water:** {week_data['water_liters']:.1f}L ({daily_water:.1f}L/day)
  - 🍎 **Fruits:** {week_data['fruits_grams']:.0f}g | 🥬 **Vegetables:** {week_data['vegetables_grams']:.0f}g
  - 🫘 **Legumes:** {week_data['legumes_grams']:.0f}g | 🍗 **Meat:** {week_data['meat_grams']:.0f}g
  - 🌾 **Grains:** {week_data['grains_grams']:.0f}g | 🥜 **Nuts/Seeds:** {week_data['nuts_seeds_grams']:.0f}g

  ---

  ## 🤖 AI Recommendations
  *Personalized based on your data, preferences, allergies, and custom targets\n*

  {recommendation}
  """

    def load_user_targets(self, user_id):
        """Load user targets and format for display in main tab"""
        try:
            if not user_id or not user_id.strip():
                return "❌ Please enter a valid User ID"

            targets = self.get_user_targets(user_id.strip())

            return f"""# 📊 Your Current Targets Loaded

## 🎯 Targets for {user_id}:

### 🏃‍♂️ Activity Targets:
- **Weekly Steps:** {targets['weekly_steps']:,}
- **Zone Minutes:** {targets['weekly_zone_minutes']}
- **Exercise Sessions:** {targets['weekly_exercise_sessions']}

### 😴 Recovery Target:
- **Daily Sleep:** {targets['daily_sleep_hours']} hours

### 🍎 Nutrition Targets:
- **Weekly Water:** {targets['weekly_water_liters']} liters
- **Weekly Fruits:** {targets['weekly_fruits_grams']:,} grams
- **Weekly Vegetables:** {targets['weekly_vegetables_grams']:,} grams

*These targets will be used to evaluate your progress when you get AI recommendations.*

**💡 Tip:** Go to the 'Customize Targets' tab to modify these values if needed.
"""

        except Exception as e:
            return f"❌ Error loading targets: {str(e)}"

    def save_user_targets(self, user_id, weekly_steps, zone_minutes, exercise_sessions,
                          daily_sleep, weekly_water):
        """Saves user's custom targets to a file"""
        if not user_id or not user_id.strip():
            return "❌ Error: Please enter a valid User ID to save targets."

        targets_file = f"{self.wellbeing_system.data_manager.data_dir}/targets_{user_id.strip()}.json"

        try:
            targets_data = {
                'weekly_steps': int(weekly_steps) if weekly_steps else self.default_targets['weekly_steps'],
                'weekly_zone_minutes': int(zone_minutes) if zone_minutes else self.default_targets['weekly_zone_minutes'],
                'weekly_exercise_sessions': int(exercise_sessions) if exercise_sessions else self.default_targets['weekly_exercise_sessions'],
                'daily_sleep_hours': float(daily_sleep) if daily_sleep else self.default_targets['daily_sleep_hours'],
                'weekly_water_liters': float(weekly_water) if weekly_water else self.default_targets['weekly_water_liters'],
              }

            with open(targets_file, 'w') as f:
                json.dump(targets_data, f, indent=2)

            return f"✅ Targets saved successfully for user '{user_id.strip()}'."

        except Exception as e:
            return f"❌ Error saving targets: {str(e)}"

    def get_user_targets(self, user_id: str) -> Dict:
        """Loads user's custom targets, or returns defaults"""
        targets_file = f"{self.wellbeing_system.data_manager.data_dir}/targets_{user_id.strip()}.json"

        if os.path.exists(targets_file):
            try:
                with open(targets_file, 'r') as f:
                    targets = json.load(f)
                    # Ensure all keys from default are present (for backward compatibility)
                    full_targets = self.default_targets.copy()
                    full_targets.update(targets)
                    return full_targets
            except Exception as e:
                print(f"⚠️ Warning: Could not load targets for user '{user_id.strip()}': {str(e)}. Using defaults.")
                return self.default_targets
        else:
            return self.default_targets

    def load_user_targets_display(self, user_id):
        """Loads user targets and formats for display in the targets tab"""
        if not user_id or not user_id.strip():
             return "❌ Error: Please enter a valid User ID to load targets.", *[gr.Number(value=self.default_targets[key]) for key in self.default_targets] # Return defaults to update UI

        targets = self.get_user_targets(user_id.strip())

        output_message = f"✅ Loaded targets for user '{user_id.strip()}'."

        # Return the message and the target values to update the Gradio Number components
        return (
            output_message,
            targets.get('weekly_steps', self.default_targets['weekly_steps']),
            targets.get('weekly_zone_minutes', self.default_targets['weekly_zone_minutes']),
            targets.get('weekly_exercise_sessions', self.default_targets['weekly_exercise_sessions']),
            targets.get('daily_sleep_hours', self.default_targets['daily_sleep_hours']),
            targets.get('weekly_water_liters', self.default_targets['weekly_water_liters']),
            )

    def reset_to_defaults(self):
        """Resets the target input fields to default values"""
        return (
            "🎯 Resetting targets to defaults.",
            self.default_targets['weekly_steps'],
            self.default_targets['weekly_zone_minutes'],
            self.default_targets['weekly_exercise_sessions'],
            self.default_targets['daily_sleep_hours'],
            self.default_targets['weekly_water_liters'],
        )









# Main execution

In [ ]:
# Uncomment to run:
app = WellbeingApp(base_model_id="ContactDoctor/Bio-Medical-Llama-3-8B", adapter_path="AnjaliNV/WellBeing_LLM")
demo = app.create_wellbeing_app()
demo.launch(share=True, debug=True)

# Fine Tuning

In [ ]:
import os
import json
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
import torch # Import torch to check for CUDA

# Load your dataset from JSONL

new_data = [
{"input": "Demographics: Gender(F), Age(50), Height(171.2 cm), Weight(84.8 kg), BMI(28.9)\nPhysical Activity: Weekly Steps(61434), Zone Minutes per week(90), Exercise Sessions per week(3), Duration minutes (avg)(83.16666667)\nSleep: Average hours per night(6.85)\nDietary Preference: Declared Diet(Vegan), Allergies(nuts)\nWeekly Food Consumption: Dairy(0.1 L), Legumes(680 g), Meat(1455 g), Fruits(477 g), Vegetables(806 g), Grains(1189 g), Nuts(35 g), Water(16.4 L)", "output": "**1) Food Recommendation**\n- Overall Assessment: There is a contradiction in the declared diet being vegan while the weekly food consumption includes a significant amount of meat (1455 g). Additionally, the intake of dairy (0.1 L) is very low, which is consistent with a vegan diet, but the consumption of legumes is quite high. There is a potential deficiency in protein quality due to the high meat intake conflicting with the vegan declaration. The water intake is adequate, but the overall balance of macronutrients needs adjustment.\n- Areas of Improvements: [{'Legumes': 'Increase to 1000 g per week'}, {'Fruits': 'Increase to 700 g per week'}, {'Vegetables': 'Increase to 1000 g per week'}, {'Grains': 'Decrease to 800 g per week'}, {'Dairy': 'Eliminate completely to align with vegan diet'}]\n- Suggested Meals: ['Chickpea salad with quinoa, cucumber, and tomatoes', 'Lentil soup with spinach and carrots', 'Stir-fried tofu with mixed vegetables and brown rice', 'Fruit smoothie with banana, spinach, and almond milk (ensure nut-free)', 'Vegetable curry with chickpeas served with whole grain bread', 'Stuffed bell peppers with black beans, corn, and avocado', 'Oatmeal topped with berries and a splash of plant-based milk']\n\n**2) Physical Activity**\n- Activity Assessment: Your weekly steps (61,434) indicate a commendable level of daily activity, which is beneficial for overall health. However, the average of 3 exercise sessions per week with a duration of about 83 minutes each could be enhanced to better support your fitness goals. Given your dietary profile, which includes a significant amount of meat despite declaring a vegan diet, it’s important to ensure your physical activity aligns with your nutritional intake to support muscle maintenance and overall health.\n- Zone Minutes and Intensity Feedback: You currently have 90 zone minutes per week, which is a good start but could be improved. Aim to increase your zone minutes by incorporating more high-intensity interval training (HIIT) or brisk walking sessions. Try to reach at least 120 zone minutes per week by adding 30 minutes of moderate to vigorous activity on two additional days.\n- Strength/Cardio Tips: ['For strength training, aim for 2-3 sessions per week. Focus on compound movements such as: 1) Squats: 3 sets of 10-12 reps, 2) Push-ups: 3 sets of 8-10 reps, 3) Dumbbell rows: 3 sets of 10-12 reps per side, 4) Plank: 3 sets of 30-60 seconds. For cardio, include activities like brisk walking, cycling, or swimming for at least 150 minutes a week.']\n- Weekly Goals: ['Increase weekly steps to 70,000 steps.', 'Increase exercise sessions to 4 per week.', 'Increase zone minutes to 120 per week.']\n\n**3) Sleep & Well-being**\n- Sleep Review: Your average sleep duration of 6.85 hours is slightly below the recommended 7-9 hours for optimal health, which can impact recovery and overall well-being. The intake of complex carbohydrates is relatively low due to the high grain consumption and potential dairy elimination, which may affect serotonin production and sleep quality. Additionally, if you consume caffeine later in the day, this could disrupt your sleep patterns. Ensuring adequate hydration is important, but aim to avoid excessive fluid intake right before bed to minimize nighttime awakenings.\n- Suggestions: ['Aim to establish a consistent sleep schedule by going to bed and waking up at the same time each day.', 'Incorporate a pre-sleep routine that includes relaxing activities such as reading or meditation, and avoid screens one hour before bedtime.', 'Consider adjusting your dinner to include more complex carbohydrates (like whole grains or starchy vegetables) to promote better sleep.', 'Limit caffeine intake to earlier in the day and consider reducing overall consumption to improve sleep quality.', \"Ensure you're hydrating throughout the day, but reduce fluid intake in the hour leading up to bedtime to minimize disruptions.\"]\n\n**4) Weekly Summary**\n- Summary: This week, focus on aligning your dietary choices with your vegan declaration by eliminating meat and dairy while increasing your intake of legumes, fruits, and vegetables. Enhance your physical activity by incorporating more high-intensity workouts and increasing your exercise sessions to four per week. Prioritize sleep by establishing a consistent schedule and incorporating relaxing pre-sleep ro...\n- Goals: ['Eliminate all meat and dairy from your diet.', 'Increase weekly fruit intake to 700 g and vegetable intake to 1000 g.', 'Increase exercise sessions to 4 per week and zone minutes to 120.', 'Establish a consistent sleep schedule, aiming for at least 7 hours of sleep per night.', 'Incorporate a relaxing pre-sleep routine and limit caffeine intake after midday.']"},
{"input": "Demographics: Gender(M), Age(27), Height(187.8 cm), Weight(78.7 kg), BMI(22.2)\nPhysical Activity: Weekly Steps(34774), Zone Minutes per week(114), Exercise Sessions per week(2), Duration minutes (avg)(70.25)\nSleep: Average hours per night(7.175)\nDietary Preference: Declared Diet(Vegetarian), Allergies(nuts)\nWeekly Food Consumption: Dairy(2.8 L), Legumes(546 g), Meat(528 g), Fruits(236 g), Vegetables(1500 g), Grains(2458 g), Nuts(122 g), Water(11.0 L)", "output": "**1) Food Recommendation**\n- Overall Assessment: The reported food consumption shows a contradiction as the declared diet is vegetarian, yet there is a significant intake of meat (528 g), which is not aligned with a vegetarian lifestyle. Additionally, the consumption of legumes is relatively low compared to grains, and fruit intake is below recommended levels. There is also an inconsistency with nut intake since the individual has a nut allergy but reported consuming 122 g of nuts weekly.\n- Areas of Improvements: [{'Legumes': 'Target 800 g per week'}, {'Fruits': 'Target 500 g per week'}, {'Nuts': 'Eliminate completely due to allergy'}, {'Vegetables': 'Maintain or increase slightly to 2000 g per week'}, {'Water': 'Target 12 L per week'}]\n- Suggested Meals: ['Chickpea salad with mixed vegetables and a lemon vinaigrette', 'Quinoa and black bean stuffed peppers', 'Vegetable stir-fry with tofu and brown rice', 'Lentil soup with spinach and whole grain bread', 'Fruit smoothie with spinach, banana, and plant-based milk', 'Oatmeal topped with berries and seeds (avoid nuts)', 'Vegetable curry with chickpeas served with whole grain naan']\n\n**2) Physical Activity**\n- Activity Assessment: Your weekly steps (34,774) indicate a strong foundation in daily movement, which is excellent for cardiovascular health. However, with only 2 exercise sessions per week averaging 70.25 minutes, there's an opportunity to enhance your structured fitness routine. Given your dietary inconsistencies, particularly with meat consumption, ensure you’re fueling your body appropriately to support muscle recovery and overall energy levels.\n- Zone Minutes and Intensity Feedback: Currently, you have 114 zone minutes per week. To improve cardiovascular fitness, aim to increase this to 150 minutes per week. Consider incorporating higher intensity intervals during your sessions to elevate your heart rate and maximize calorie burn. You can achieve this by adding short bursts of high-intensity activity followed by recovery periods.\n- Strength/Cardio Tips: ['Strength Training: Aim for 2-3 sessions per week focusing on major muscle groups. Example: 3 sets of 10-12 reps of squats, lunges, push-ups, and dumbbell rows.', 'Cardio: Include 3-4 sessions of cardiovascular workouts per week. Example: 30 minutes of cycling, running, or swimming at moderate to high intensity, gradually increasing duration or intensity.']\n- Weekly Goals: ['Increase weekly steps to 40,000.', 'Increase exercise sessions to 4 per week.', 'Increase zone minutes to 150.']\n\n**3) Sleep & Well-being**\n- Sleep Review: Your average sleep of 7.175 hours is within the recommended range, but there may be factors affecting sleep quality. Consider your meal timing, especially if consuming caffeine or heavy meals close to bedtime, which can disrupt sleep. Ensure you're consuming enough carbohydrates to support recovery, particularly after exercise, as this can improve sleep quality. Additionally, a high intake of vegetables and hydration is beneficial, but avoid consuming excessive fluids right before bed to prevent disruptions during the night.\n- Suggestions: ['Aim to finish eating at least 2-3 hours before bedtime to improve digestion and sleep quality.', \"Incorporate a pre-sleep routine that includes winding down activities such as reading or gentle stretching to signal your body it's time to relax.\", 'Consider adjusting your carbohydrate intake post-workout to replenish glycogen stores, which can aid recovery and promote better sleep.', 'Maintain hydration throughout the day, but reduce fluid intake in the hour leading up to bedtime to minimize nighttime awakenings.']\n\n**4) Weekly Summary**\n- Summary: This week, focus on aligning your diet with your vegetarian preference by eliminating meat and nuts, while increasing your intake of legumes and fruits. Enhance your physical activity by adding two more structured exercise sessions and increasing your weekly steps. Prioritize sleep quality by adjusting meal timing and incorporating a calming pre-sleep routine.\n- Goals: ['Eliminate meat and nuts from your diet.', 'Increase weekly steps to 40,000.', 'Increase exercise sessions to 4 per week.', 'Consume 800 g of legumes and 500 g of fruits weekly.', 'Finish eating 2-3 hours before bedtime to improve sleep quality.']"},
{"input": "Demographics: Gender(M), Age(42), Height(180.1 cm), Weight(65.4 kg), BMI(20.1)\nPhysical Activity: Weekly Steps(27846), Zone Minutes per week(35), Exercise Sessions per week(2), Duration minutes (avg)(72.0)\nSleep: Average hours per night(7.266666667)\nDietary Preference: Declared Diet(Mediterranean), Allergies(none)\nWeekly Food Consumption: Dairy(1.8 L), Legumes(443 g), Meat(993 g), Fruits(1148 g), Vegetables(918 g), Grains(760 g), Nuts(192 g), Water(8.0 L)", "output": "**1) Food Recommendation**\n- Overall Assessment: The individual's reported meat intake is quite high compared to other protein sources like legumes and nuts. While dairy and vegetables are consumed in moderate amounts, the intake of nuts is relatively low for a Mediterranean diet. Additionally, while fruit intake appears adequate, the range of fruits could be more diverse. Overall, there seems to be a reliance on meat as the primary protein source, which may not align with a balanced Mediterranean diet.\n- Areas of Improvements: [{'low_intake_group': 'Nuts', 'weekly_target': '400 g'}, {'low_intake_group': 'Variety of Fruits', 'weekly_target': '1500 g'}, {'low_intake_group': 'Whole Grains', 'weekly_target': '1000 g'}]\n- Suggested Meals: ['Breakfast: Greek yogurt with honey, mixed berries, and a handful of walnuts.', 'Lunch: Quinoa salad with chickpeas, diced cucumber, tomatoes, parsley, and a lemon-olive oil dressing.', 'Snack: Hummus with carrot and cucumber sticks.', 'Dinner: Grilled salmon with a side of sautéed spinach and a small serving of whole grain pasta.', 'Dessert: Fresh fruit salad with a mix of seasonal fruits, topped with a sprinkle of nuts.']\n\n**2) Physical Activity**\n- Activity Assessment: You are already quite active, averaging over 27,000 steps per week, which is excellent for cardiovascular health. Your exercise sessions are limited to two per week; increasing this frequency could enhance overall fitness. The average duration of 72 minutes per session is commendable but ensure that your workouts include a mix of strength and cardio to balance your high meat intake with muscle recovery and growth.\n- Zone Minutes and Intensity Feedback: Your current zone minutes are low at 35 per week. To raise intensity, consider incorporating interval training into your sessions, aiming for at least 150 minutes of moderate to vigorous activity weekly. This could include activities like cycling, swimming, or higher-intensity strength training, which can help improve cardiovascular health and muscle strength.\n- Strength/Cardio Tips: ['For strength training, aim for 2-3 sessions per week focusing on major muscle groups. Example: 3 sets of 10-12 reps of squats, lunges, push-ups, and rows. Include core exercises like planks and dead bugs for stability.', 'For cardio, try to incorporate at least 30 minutes of moderate-intensity exercise (like brisk walking or cycling) 5 times a week, or 25 minutes of vigorous activity (like running or high-intensity interval training) 3 times a week.']\n- Weekly Goals: ['Increase weekly steps to 30,000.', 'Increase exercise sessions to 3-4 per week.', 'Aim for 150 minutes of zone minutes per week.']\n\n**3) Sleep & Well-being**\n- Sleep Review: Your average sleep duration of approximately 7.3 hours is within the recommended range for adults. However, ensure that your sleep quality is not compromised by factors such as late caffeine consumption or inadequate carbohydrate intake, which can affect energy levels and recovery. As your physical activity and exercise intensity increase, proper nutrition timing and hydration will be crucial for optimizing sleep and recovery.\n- Suggestions: ['Consider reducing caffeine intake in the afternoon and evening to improve sleep quality.', 'Incorporate a small carbohydrate-rich snack before bedtime, such as whole grain toast with a bit of nut butter, to support recovery and promote better sleep.', \"Establish a calming pre-sleep routine, such as reading or meditation, to signal to your body that it's time to wind down.\", 'Ensure proper hydration throughout the day, but limit water intake close to bedtime to avoid disruptions during the night.']\n\n**4) Weekly Summary**\n- Summary: This week, focus on diversifying your diet by increasing fruit and nut intake while reducing reliance on meat. Enhance your physical activity by adding an extra exercise session and incorporating interval training to boost cardiovascular health. Prioritize sleep quality by adjusting caffeine intake and establishing a calming pre-sleep routine.\n- Goals: ['Increase fruit intake to 1500 g and nuts to 400 g this week.', 'Add one more exercise session for a total of 3-4 per week.', 'Incorporate at least 150 minutes of moderate to vigorous activity.', 'Establish a calming pre-sleep routine and reduce afternoon caffeine.', 'Include a small carbohydrate-rich snack before bedtime.']"},
{"input": "Demographics: Gender(M), Age(26), Height(165.9 cm), Weight(57.3 kg), BMI(20.7)\nPhysical Activity: Weekly Steps(36069), Zone Minutes per week(47), Exercise Sessions per week(3), Duration minutes (avg)(67.0)\nSleep: Average hours per night(7.325)\nDietary Preference: Declared Diet(Vegetarian), Allergies(nuts)\nWeekly Food Consumption: Dairy(0.4 L), Legumes(459 g), Meat(559 g), Fruits(1444 g), Vegetables(1025 g), Grains(853 g), Nuts(158 g), Water(10.4 L)", "output": "**1) Food Recommendation**\n- Overall Assessment: The profile indicates a vegetarian diet, but the reported meat consumption is 559 g per week, which contradicts the declared dietary preference. Additionally, the intake of dairy is quite low (0.4 L), which may lead to deficiencies in calcium and vitamin D. The intake of fruits and vegetables is notably high, which is positive, but legumes should be increased to meet protein needs in a vegetarian diet. The intake of nuts is also high, though the individual has a nut allergy, so this could pose a risk.\n- Areas of Improvements: [{'Legumes': 'Increase to 800 g per week'}, {'Dairy': 'Increase to 1 L per week'}, {'Nuts': 'Avoid completely due to allergy'}]\n- Suggested Meals: ['Chickpea salad with mixed vegetables and a yogurt dressing', 'Lentil stew with whole grain bread', 'Vegetable stir-fry with tofu and quinoa', 'Smoothie with spinach, banana, and dairy milk', 'Vegetable curry with brown rice and dairy-free yogurt']\n\n**2) Physical Activity**\n- Activity Assessment: Your current activity level is commendable, with an average of 36,069 steps per week and 3 exercise sessions averaging 67 minutes each. However, your zone minutes are relatively low at 47 per week, indicating that while you are active, there may be opportunities to increase the intensity of your workouts to enhance cardiovascular fitness and overall health. Your vegetarian diet, particularly with low dairy intake, may impact your energy levels, so ensure you are fueling adequately for your activity.\n- Zone Minutes and Intensity Feedback: To raise your zone minutes, consider incorporating more high-intensity interval training (HIIT) sessions or increasing the intensity of your current workouts. Aim for at least 150 minutes of moderate to vigorous activity per week, which could mean increasing your workout durations or adding a couple of high-intensity sessions. For example, you could add a 20-minute HIIT session twice a week.\n- Strength/Cardio Tips: ['Include strength training 2-3 times a week focusing on all major muscle groups. Example exercises: 3 sets of 10-12 reps of squats, lunges, and push-ups. For cardio, aim for 30 minutes of running or cycling at a moderate pace, or 15-20 minutes of interval training.', 'Incorporate bodyweight exercises like planks and burpees for overall strength and cardiovascular benefits.']\n- Weekly Goals: ['Increase weekly steps to 40,000.', 'Aim for 4 exercise sessions per week.', 'Boost zone minutes to at least 100.']\n\n**3) Sleep & Well-being**\n- Sleep Review: Your average sleep duration of 7.325 hours is within the recommended range for your age group, which is beneficial for recovery. However, it's important to consider how your diet impacts your sleep quality. Low dairy intake may lead to insufficient calcium and vitamin D, which can affect sleep quality. Additionally, if you consume caffeine later in the day, it could disrupt your sleep. Ensure that you are consuming enough carbohydrates in the evening to promote serotonin production, which aids in sleep. Lastly, if your workouts are intense, consider having a post-exercise meal or snack to replenish energy stores, which can also improve sleep quality.\n- Suggestions: ['Aim to finish any caffeine consumption by early afternoon to prevent sleep disruption.', 'Consider including a balanced evening meal rich in carbohydrates and protein to support recovery and promote better sleep.', 'Establish a pre-sleep routine that includes winding down activities and minimizing screen time at least an hour before bed.', 'Stay hydrated throughout the day but reduce fluid intake in the hour leading up to bedtime to minimize nighttime awakenings.']\n\n**4) Weekly Summary**\n- Summary: This week, focus on aligning your diet with your vegetarian preference by increasing legume and dairy intake while avoiding nuts due to your allergy. Enhance your activity by incorporating higher intensity workouts to boost your zone minutes, aiming for at least 100 per week. Maintain your good sleep habits by ensuring adequate evening nutrition and managing caffeine intake.\n- Goals: ['Increase legume intake to 800 g and dairy to 1 L this week.', 'Add 2 high-intensity interval training sessions to your weekly routine.', 'Increase weekly steps to 40,000.', 'Establish a pre-sleep routine to improve sleep quality.', 'Finish caffeine consumption by early afternoon.']"},
{"input": "Demographics: Gender(M), Age(43), Height(160.5 cm), Weight(50.3 kg), BMI(19.4)\nPhysical Activity: Weekly Steps(27714), Zone Minutes per week(64), Exercise Sessions per week(0), Duration minutes (avg)(77.5)\nSleep: Average hours per night(6.15)\nDietary Preference: Declared Diet(Mediterranean), Allergies(nuts)\nWeekly Food Consumption: Dairy(1.1 L), Legumes(294 g), Meat(826 g), Fruits(396 g), Vegetables(1576 g), Grains(1045 g), Nuts(11 g), Water(8.0 L)", "output": "**1) Food Recommendation**\n- Overall Assessment: The reported intake shows a low consumption of fruits and vegetables relative to the recommended amounts for a Mediterranean diet. While the vegetable intake is relatively high, the fruit intake is below ideal levels, which should be increased for better micronutrient balance. Additionally, the protein sources seem to be high in meat intake without sufficient balance from plant-based proteins or fish, which are staples in a Mediterranean diet. There is a very low intake of nuts, which should be avoided in excess due to the allergy but should still be considered for variety in protein sources if alternatives are available.\n- Areas of Improvements: [{'Fruits': 'Increase to 700 g per week'}, {'Fish/Seafood': 'Introduce at least 200 g per week'}, {'Legumes': 'Aim for 500 g per week'}, {'Nuts': 'Consider alternatives with similar nutritional profiles, aiming for 30 g per week'}, {'Whole Grains': 'Increase to 1400 g per week'}]\n- Suggested Meals: ['Breakfast: Greek yogurt with honey and mixed berries', 'Lunch: Quinoa salad with chickpeas, cucumbers, tomatoes, and olive oil dressing', 'Dinner: Grilled salmon with a side of roasted vegetables and a small portion of whole grain pasta', 'Snack: Sliced apples with a sunflower seed butter alternative (if nut allergy is a concern)', 'Dessert: Fresh fruit salad with a drizzle of balsamic vinegar']\n\n**2) Physical Activity**\n- Activity Assessment: Your weekly steps are impressive at 27,714, indicating a strong base of daily activity. However, you are currently not engaging in structured exercise sessions, which could enhance your fitness levels and overall health. Incorporating strength training and cardiovascular workouts can help balance your activity profile, especially considering your high protein intake from meat.\n- Zone Minutes and Intensity Feedback: With 64 zone minutes per week, consider increasing the intensity of your activities to elevate your heart rate more regularly. Aim for at least 150 minutes of moderate-intensity aerobic activity each week, which can be achieved through brisk walking, cycling, or swimming. Gradually increase your zone minutes to around 120 per week.\n- Strength/Cardio Tips: ['Strength Training: Aim for 2-3 sessions per week. Focus on full-body workouts. Example: 3 sets of 10-12 reps of squats, lunges, push-ups, and rows. Incorporate resistance bands or light weights.', 'Cardio: Include 150 minutes of moderate-intensity cardio per week. Example: 30 minutes of brisk walking 5 times a week or cycling for 20 minutes 3 times a week.']\n- Weekly Goals: ['Increase weekly steps to 30,000.', 'Incorporate 2 structured exercise sessions per week.', 'Raise zone minutes to 120 per week.']\n\n**3) Sleep & Well-being**\n- Sleep Review: Your average sleep duration of 6.15 hours is below the recommended 7-9 hours for optimal recovery and overall health. This could be influenced by your dietary habits, such as low carbohydrate intake which may affect sleep quality, and potential late caffeine consumption if present. Additionally, the lack of structured exercise may lead to insufficient physical fatigue, impacting your ability to fall asleep. It's essential to create a balanced pre-sleep routine and ensure proper nutrient timing to support better sleep.\n- Suggestions: ['Aim to establish a consistent sleep schedule by going to bed and waking up at the same time every day.', 'Consider incorporating a carbohydrate-rich snack before bed, such as whole grain toast with a bit of honey or yogurt with fruit, to promote serotonin production and improve sleep quality.', 'Limit caffeine intake after mid-afternoon to minimize its impact on sleep onset.', \"Create a pre-sleep routine that includes relaxation techniques, such as reading or gentle stretching, to signal to your body that it's time to wind down.\", 'Stay hydrated throughout the day, but reduce fluid intake in the hour leading up to sleep to minimize nighttime awakenings.']\n\n**4) Weekly Summary**\n- Summary: This week, focus on enhancing your Mediterranean diet by increasing fruit intake to 700g and incorporating at least 200g of fish or seafood. Add structured exercise sessions, aiming for 2-3 sessions of strength training and increase your zone minutes to 120 per week. Prioritize sleep by establishing a consistent sleep schedule and aiming for at least 7 hours of rest each night.\n- Goals: ['Increase fruit intake to 700g and introduce 200g of fish/seafood this week.', 'Incorporate 2 structured exercise sessions, focusing on strength training.', 'Increase weekly steps to 30,000 and zone minutes to 120.', 'Establish a consistent sleep schedule, aiming for 7+ hours of sleep each night.', 'Include a carbohydrate-rich snack before bed to improve sleep quality.']"},
{"input": "Demographics: Gender(M), Age(45), Height(181.6 cm), Weight(70.7 kg), BMI(21.4)\nPhysical Activity: Weekly Steps(41387), Zone Minutes per week(100), Exercise Sessions per week(3), Duration minutes (avg)(74.71428571)\nSleep: Average hours per night(6.871428571)\nDietary Preference: Declared Diet(Vegan), Allergies(none)\nWeekly Food Consumption: Dairy(0.6 L), Legumes(759 g), Meat(125 g), Fruits(1615 g), Vegetables(1541 g), Grains(2315 g), Nuts(66 g), Water(17.0 L)", "output": "**1) Food Recommendation**\n- Overall Assessment: The individual follows a vegan diet but consumes a small amount of meat (125 g), which contradicts the declared diet. Additionally, the consumption of nuts is low compared to other food groups. The intake of legumes is high, but diversity in protein sources could be improved. The overall intake of grains is also significant, which may lead to an imbalance if not paired with sufficient protein and healthy fats.\n- Areas of Improvements: [{'low_intake_group': 'Nuts', 'concrete_weekly_target': '200 g'}, {'low_intake_group': 'Legumes', 'concrete_weekly_target': '1000 g'}, {'low_intake_group': 'Whole Grains', 'concrete_weekly_target': '1500 g'}, {'low_intake_group': 'Fruits', 'concrete_weekly_target': '2000 g'}]\n- Suggested Meals: ['Chickpea salad with mixed greens, cherry tomatoes, cucumbers, and a tahini dressing', 'Quinoa bowl with black beans, avocado, corn, salsa, and lime', 'Smoothie with spinach, banana, almond milk, and a scoop of nut butter', 'Stir-fried tofu with broccoli, bell peppers, and brown rice', 'Overnight oats with almond milk, chia seeds, berries, and a sprinkle of nuts', 'Lentil soup with carrots, celery, and whole-grain bread on the side', 'Roasted vegetable and hummus wrap with whole grain tortilla']\n\n**2) Physical Activity**\n- Activity Assessment: Your weekly step count of 41,387 is impressive, indicating a strong foundation of daily activity. With 3 exercise sessions averaging about 75 minutes each, you're maintaining a good balance. However, consider the potential impact of your diet on recovery and performance, particularly with lower protein and healthy fat intake from nuts.\n- Zone Minutes and Intensity Feedback: With 100 zone minutes per week, you may want to consider increasing the intensity of your workouts or adding an additional session to enhance cardiovascular fitness. Aim for at least 150 zone minutes per week, which can be achieved through interval training or longer moderate-intensity sessions.\n- Strength/Cardio Tips: ['Incorporate strength training exercises like squats, deadlifts, and push-ups 2-3 times a week. Aim for 3 sets of 8-12 reps for each exercise.', 'Add cardio workouts such as cycling or running for 30 minutes at a moderate pace, 2-3 times a week, to increase cardiovascular endurance.']\n- Weekly Goals: ['Increase weekly steps to 50,000.', 'Add 1 extra exercise session for a total of 4 sessions.', 'Raise zone minutes to 150 per week.']\n\n**3) Sleep & Well-being**\n- Sleep Review: Your average sleep duration of 6.87 hours is slightly below the recommended 7-9 hours for optimal recovery. A potential risk could be late caffeine consumption, which may interfere with sleep quality. Additionally, your diet may lack sufficient carbohydrates, particularly if the intake of fruits is low, which can impact energy levels and recovery from training. Ensuring a balanced intake of macronutrients, particularly before sleep, is crucial for enhancing sleep quality and recovery.\n- Suggestions: ['Aim to consume a carbohydrate-rich snack, such as a smoothie or overnight oats, 1-2 hours before bedtime to promote better sleep.', \"Establish a consistent sleep schedule by going to bed and waking up at the same time each day to regulate your body's internal clock.\", 'Limit caffeine intake after 2 PM to minimize its impact on sleep quality.', \"Create a calming pre-sleep routine that includes activities like reading, stretching, or meditation to signal to your body that it's time to wind down.\", 'Ensure adequate hydration throughout the day, but reduce fluid intake in the hour leading up to bedtime to minimize nighttime awakenings.']\n\n**4) Weekly Summary**\n- Summary: This week, focus on enhancing your vegan diet by increasing your intake of nuts and fruits while maintaining a high level of legumes and whole grains. Incorporate an additional exercise session to boost your cardiovascular fitness and aim for a total of 150 zone minutes. Prioritize sleep by establishing a consistent schedule and consuming a carbohydrate-rich snack before bedtime to improve recover...\n- Goals: ['Increase nut intake to 200 g this week.', 'Add 1 extra exercise session for a total of 4 sessions.', 'Raise weekly zone minutes to 150.', 'Consume 2000 g of fruits throughout the week.', 'Establish a consistent sleep schedule and limit caffeine after 2 PM.']"},
{"input": "Demographics: Gender(M), Age(44), Height(181.5 cm), Weight(86.9 kg), BMI(26.3)\nPhysical Activity: Weekly Steps(17966), Zone Minutes per week(143), Exercise Sessions per week(0), Duration minutes (avg)(80.0)\nSleep: Average hours per night(7.8)\nDietary Preference: Declared Diet(No Preference), Allergies(nuts)\nWeekly Food Consumption: Dairy(4.5 L), Legumes(567 g), Meat(744 g), Fruits(159 g), Vegetables(439 g), Grains(2078 g), Nuts(34 g), Water(4.2 L)", "output": "**1) Food Recommendation**\n- Overall Assessment: The individual has a BMI of 26.3, indicating overweight status. While the weekly step count is high, there are no structured exercise sessions, which may affect overall fitness. The intake of fruits (159 g) is below recommended levels, and dairy consumption (4.5 L) is quite high, potentially leading to excessive calorie intake and fat. Legumes (567 g) are consumed adequately, but meat intake (744 g) is on the higher side, which may contribute to an imbalanced diet. The intake of vegetables (439 g) is reasonable, but it could be improved, and grains (2078 g) are likely contributing to excess carbohydrate intake. The individual has a nut allergy, so alternative healthy fats need to be considered.\n- Areas of Improvements: [{'low_intake_group': 'Fruits', 'weekly_target': '300 g'}, {'low_intake_group': 'Vegetables', 'weekly_target': '800 g'}, {'low_intake_group': 'Whole Grains', 'weekly_target': '1500 g'}]\n- Suggested Meals: ['Breakfast: Oatmeal topped with sliced bananas and a sprinkle of cinnamon.', 'Lunch: Quinoa salad with chickpeas, diced cucumber, bell peppers, and a lemon vinaigrette.', 'Dinner: Grilled chicken breast with a side of steamed broccoli and sweet potato.', 'Snacks: Greek yogurt with berries, carrot sticks with hummus.', 'Smoothie: Spinach, banana, and almond milk smoothie for an afternoon boost.']\n\n**2) Physical Activity**\n- Activity Assessment: Your high weekly step count of 17,966 is a significant strength, indicating good baseline activity. However, the absence of structured exercise sessions means you may not be fully utilizing your potential for fitness improvement. Incorporating regular exercise can enhance your cardiovascular health, muscle strength, and overall well-being, especially given your BMI of 26.3, which indicates overweight status.\n- Zone Minutes and Intensity Feedback: To increase your zone minutes from 143 per week, aim for more intense workouts. Consider incorporating interval training or higher intensity sessions 2-3 times a week to boost your heart rate and calorie burn. Gradually increase your duration to 30-45 minutes per session, focusing on maintaining a heart rate within your target zone.\n- Strength/Cardio Tips: ['Strength Training: Aim for 2-3 sessions per week. Exercises like squats (3 sets of 10-15 reps), push-ups (3 sets of 8-12 reps), and lunges (3 sets of 10-12 reps per leg) will build muscle and improve metabolism.', 'Cardio: Incorporate 150 minutes of moderate aerobic activity weekly, such as brisk walking or cycling. Start with 20-30 minutes sessions, 5 days a week, and gradually increase duration or intensity.']\n- Weekly Goals: ['Increase weekly exercise sessions to 3-4 times per week.', 'Aim for 200 minutes of zone minutes by increasing intensity in workouts.', 'Continue to maintain or increase weekly step count to 20,000 steps.']\n\n**3) Sleep & Well-being**\n- Sleep Review: Your average sleep duration of 7.8 hours is within the recommended range, which is beneficial for recovery and overall health. However, the high dairy consumption may lead to digestive issues, potentially impacting sleep quality. Additionally, if you consume caffeine late in the day, it could interfere with your ability to fall asleep. It's crucial to ensure your carbohydrate intake is balanced throughout the day, as low carb intake in the evening can affect sleep quality and recovery. Monitoring these factors can help improve your sleep hygiene.\n- Suggestions: ['Limit dairy intake in the evening to avoid potential digestive discomfort.', \"Avoid caffeine after 2 PM to ensure it doesn't affect your sleep onset.\", 'Incorporate a carbohydrate-rich snack, like whole-grain toast with avocado or a banana, about 30-60 minutes before bedtime to promote better sleep.', \"Establish a calming pre-sleep routine, such as reading or meditation, to signal your body that it's time to wind down.\", 'Maintain hydration throughout the day, but reduce fluid intake an hour before bed to minimize nighttime awakenings.']\n\n**4) Weekly Summary**\n- Summary: This week, focus on enhancing your diet by increasing your fruit and vegetable intake while moderating dairy and meat consumption. Incorporate structured exercise sessions 3-4 times a week to improve cardiovascular health and muscle strength. Maintain your high step count and ensure a balanced carbohydrate intake in the evening to support better sleep quality.\n- Goals: ['Increase fruit intake to 300 g and vegetables to 800 g by the end of...', 'Incorporate 3-4 structured exercise sessions, including strength training and cardio.', 'Aim for 200 zone minutes by increasing workout intensity.', 'Limit dairy intake in the evening and avoid caffeine after 2 PM.', 'Establish a calming pre-sleep routine and monitor carbohydrate intake before bedtime.']"},
{"input": "Demographics: Gender(F), Age(22), Height(167.1 cm), Weight(66.4 kg), BMI(23.7)\nPhysical Activity: Weekly Steps(15573), Zone Minutes per week(44), Exercise Sessions per week(1), Duration minutes (avg)(91.5)\nSleep: Average hours per night(6.0)\nDietary Preference: Declared Diet(No Preference), Allergies(eggs)\nWeekly Food Consumption: Dairy(3.8 L), Legumes(241 g), Meat(751 g), Fruits(227 g), Vegetables(399 g), Grains(965 g), Nuts(118 g), Water(5.0 L)", "output": "**1) Food Recommendation**\n- Overall Assessment: The individual has a balanced diet overall but shows a high intake of grains and meat, while fruits and vegetables could be increased. The dairy intake is on the higher end, and the individual has a low consumption of nuts. Sleep duration is also below the recommended level for optimal health.\n- Areas of Improvements: [{'food_group': 'Fruits', 'weekly_target': '500 g'}, {'food_group': 'Vegetables', 'weekly_target': '700 g'}, {'food_group': 'Nuts', 'weekly_target': '200 g'}, {'food_group': 'Sleep', 'weekly_target': '7-9 hours per night'}]\n- Suggested Meals: ['Breakfast: Overnight oats with chia seeds, banana, and almond milk', 'Snack: Greek yogurt (dairy-free) with mixed berries and a handful of walnuts', 'Lunch: Quinoa salad with black beans, corn, diced bell peppers, and a squeeze of lime', 'Snack: Hummus with carrot and cucumber sticks', 'Dinner: Grilled chicken or tofu with steamed broccoli, sweet potatoes, and a side salad with mixed greens', 'Dessert: Chia pudding made with almond milk topped with fresh fruit and a sprinkle of almonds']\n\n**2) Physical Activity**\n- Activity Assessment: You have a commendable weekly step count of 15,573, indicating a high level of daily activity. However, with only one exercise session per week averaging 91.5 minutes, there's room for improvement in structured physical activity. A more balanced approach that includes both strength and cardiovascular exercises could enhance your fitness further, especially considering your dietary intake.\n- Zone Minutes and Intensity Feedback: Your current zone minutes per week are at 44, which is relatively low. Aim to increase this by incorporating more high-intensity interval training (HIIT) or moderate-intensity workouts into your routine. Gradually increase your duration in the target heart rate zones to 60-90 minutes per week, focusing on activities that elevate your heart rate.\n- Strength/Cardio Tips: ['Incorporate strength training 2-3 times per week, focusing on major muscle groups. Recommended exercises include: \\n- Squats: 3 sets of 12-15 reps \\n- Push-ups: 3 sets of 10-12 reps \\n- Lunges: 3 sets of 12-15 reps per leg \\n- Plank: 3 sets of 30-60 seconds \\n- Dumbbell rows: 3 sets of 12-15 reps per arm.', 'For cardio, consider adding 2-3 sessions of 20-30 minutes of HIIT workouts or brisk walking/jogging to increase your zone minutes.']\n- Weekly Goals: ['Increase weekly steps to 17,000 steps.', 'Add 2 additional exercise sessions per week.', 'Raise zone minutes to 90 minutes per week.']\n\n**3) Sleep & Well-being**\n- Sleep Review: Your average sleep duration of 6.0 hours is below the recommended 7-9 hours for optimal health. Insufficient sleep can negatively impact recovery, mood, and overall performance, especially considering your high activity level. Late caffeine consumption or a diet low in complex carbohydrates could be affecting your ability to fall asleep. Ensuring proper nutrition timing, particularly with carbohydrates before bed, may help improve sleep quality.\n- Suggestions: ['Aim to establish a consistent sleep schedule by going to bed and waking up at the same time every day.', 'Limit caffeine intake in the afternoon and evening to reduce its impact on sleep onset.', \"Incorporate a pre-sleep routine that includes relaxing activities such as reading or gentle stretching to signal your body it's time to wind down.\", 'Consider having a carbohydrate-rich snack before bed, such as whole grain toast with almond butter or a small bowl of oatmeal, to help promote sleep.', 'Stay well-hydrated throughout the day but reduce fluid intake in the hour before sleep to minimize nighttime awakenings.']\n\n**4) Weekly Summary**\n- Summary: This week, focus on enhancing your diet by increasing fruit and vegetable intake while incorporating more nuts. Aim to improve your sleep duration to the recommended 7-9 hours by establishing a consistent sleep schedule and reducing caffeine intake. Additionally, increase your physical activity by adding two more exercise sessions and targeting higher intensity workouts to boost your zone minutes.\n- Goals: ['Consume at least 500g of fruits and 700g of vegetables this week.', 'Increase sleep duration to 7-9 hours per night.', 'Add 2 additional exercise sessions focusing on strength training and HIIT.', 'Raise weekly step count to 17,000 steps.', 'Achieve 90 minutes of zone minutes through increased cardio activity.']"},
{"input": "Demographics: Gender(M), Age(37), Height(176.1 cm), Weight(64.2 kg), BMI(20.6)\nPhysical Activity: Weekly Steps(13477), Zone Minutes per week(101), Exercise Sessions per week(0), Duration minutes (avg)(68.5)\nSleep: Average hours per night(6.4)\nDietary Preference: Declared Diet(Keto), Allergies(none)\nWeekly Food Consumption: Dairy(2.3 L), Legumes(330 g), Meat(695 g), Fruits(1649 g), Vegetables(986 g), Grains(1010 g), Nuts(169 g), Water(5.7 L)", "output": "**1) Food Recommendation**\n- Overall Assessment: The profile indicates a declared keto diet, which typically limits high-carb foods like grains and fruits. However, the reported intake shows a high consumption of fruits (1649 g) and grains (1010 g), contradicting the principles of a keto diet. Additionally, while meat intake is sufficient, the overall balance may not support the keto objectives due to high carb intake from fruits and grains. Dairy intake appears relatively moderate but could be adjusted for keto. Sleep duration is below the recommended amount, potentially impacting recovery and overall health.\n- Areas of Improvements: [{'low_intake_group': 'Healthy fats', 'weekly_target': 'Increase healthy fats to 700 g'}, {'low_intake_group': 'Low-carb vegetables', 'weekly_target': 'Increase low-carb vegetables to 1500 g'}]\n- Suggested Meals: ['Breakfast: Scrambled eggs cooked in butter with spinach and avocado', 'Lunch: Grilled chicken salad with mixed greens, olive oil dressing, and walnuts', 'Dinner: Baked salmon with asparagus and a side of cauliflower rice', 'Snacks: Celery sticks with almond butter, cheese slices', 'Dessert: Chia seed pudding made with unsweetened almond milk and topped with a few berries']\n\n**2) Physical Activity**\n- Activity Assessment: Your weekly steps are quite impressive at 13,477, indicating a strong base of activity. However, with zero structured exercise sessions, there is an opportunity to enhance your fitness with targeted workouts. Incorporating both strength training and cardiovascular activities can support your keto diet by promoting muscle retention and fat loss.\n- Zone Minutes and Intensity Feedback: Currently, you have 101 zone minutes per week. To improve cardiovascular fitness and support your overall health, aim to increase this to at least 150 minutes per week. You can achieve this by incorporating higher intensity workouts 2-3 times a week, along with longer walks or jogs on other days.\n- Strength/Cardio Tips: ['For strength training, aim for 2-3 sessions per week. Focus on compound movements such as: 1) Squats: 3 sets of 10-12 reps, 2) Deadlifts: 3 sets of 8-10 reps, 3) Push-ups: 3 sets to failure, 4) Bent-over rows: 3 sets of 10-12 reps.', 'For cardio, consider: 1) High-Intensity Interval Training (HIIT) 2 times a week for 20-30 minutes, 2) Steady-state cardio (like jogging or cycling) for 30-45 minutes at least once a week.']\n- Weekly Goals: ['Increase weekly steps to 15,000.', 'Incorporate 2 structured exercise sessions per week.', 'Raise zone minutes to 150 per week.']\n\n**3) Sleep & Well-being**\n- Sleep Review: Your average sleep duration of 6.4 hours is below the recommended 7-9 hours for optimal recovery and health. Insufficient sleep can lead to increased cravings for high-carb foods, potentially undermining your keto diet. Additionally, late caffeine consumption or insufficient carbohydrates (which might be affecting your energy levels) can impact sleep quality. It's essential to address these factors to improve both sleep and dietary adherence.\n- Suggestions: ['Aim to establish a consistent sleep schedule by going to bed and waking up at the same time each day.', 'Limit caffeine intake to the morning hours to prevent interference with sleep.', 'Consider a light snack before bed that includes healthy fats (like nut butter) to support your keto diet without causing a significant carb load.', 'Hydrate adequately throughout the day, but reduce liquid intake an hour before bedtime to minimize nighttime awakenings.', \"Establish a calming pre-sleep routine, such as reading or meditation, to signal your body that it's time to wind down.\"]\n\n**4) Weekly Summary**\n- Summary: This week, focus on aligning your diet with keto principles by reducing high-carb fruit and grain intake while increasing healthy fats and low-carb vegetables. Incorporate structured exercise sessions to enhance your fitness, aiming for at least 150 zone minutes. Prioritize sleep by establishing a consistent sleep schedule and creating a calming pre-sleep routine to improve recovery and overall he...\n- Goals: ['Reduce fruit intake to 500 g and grains to 200 g while increasing healthy fats...', 'Incorporate 2 structured exercise sessions per week, focusing on strength training and HIIT.', 'Increase weekly steps to 15,000.', 'Establish a consistent sleep schedule, aiming for at least 7 hours of sleep per night.', 'Limit caffeine to morning hours and create a calming pre-sleep routine.']"},
{"input": "Demographics: Gender(M), Age(36), Height(174.8 cm), Weight(50.3 kg), BMI(16.4)\nPhysical Activity: Weekly Steps(62055), Zone Minutes per week(93), Exercise Sessions per week(5), Duration minutes (avg)(42.16666667)\nSleep: Average hours per night(7.45)\nDietary Preference: Declared Diet(No Preference), Allergies(Gluten)\nWeekly Food Consumption: Dairy(4.6 L), Legumes(657 g), Meat(380 g), Fruits(1654 g), Vegetables(1911 g), Grains(1889 g), Nuts(31 g), Water(16.0 L)", "output": "**1) Food Recommendation**\n- Overall Assessment: The individual has a BMI of 16.4, indicating underweight status. The reported protein intake from meat (380 g) and legumes (657 g) seems adequate; however, the overall caloric intake is likely insufficient given the high physical activity level (62055 weekly steps and 93 zone minutes). There is a very low intake of nuts (31 g) and dairy (4.6 L) considering the need for healthy fats and calcium, especially given the gluten allergy that may limit food choices. The vegetable intake is high, but the grain intake (1889 g) may need to be reconsidered due to gluten allergy. Overall, there is a contradiction between high physical activity and low caloric intake, leading to potential nutrient deficiencies.\n- Areas of Improvements: ['Increase nut intake to 200 g per week for healthy fats', 'Increase protein sources other than meat (e.g., tofu, eggs) to 500 g per week for balanced protein intake', 'Increase dairy alternatives (e.g., almond milk) to 8 L per week for calcium and vitamin D', 'Include gluten-free grains (e.g., quinoa, rice) to maintain energy levels while avoiding gluten', 'Incorporate more legumes to reach 1 kg per week for fiber and protein']\n- Suggested Meals: ['Breakfast: Smoothie with almond milk, spinach, banana, and a scoop of protein powder', 'Lunch: Quinoa salad with chickpeas, cucumbers, tomatoes, and a lemon-tahini dressing', 'Snack: Hummus with carrot and celery sticks, and a handful of mixed nuts', 'Dinner: Stir-fried tofu with broccoli, bell peppers, and brown rice', 'Snack: Greek yogurt (or dairy-free alternative) with mixed berries and a tablespoon of honey']\n\n**2) Physical Activity**\n- Activity Assessment: You are highly active with 62055 weekly steps and 5 exercise sessions averaging 42 minutes each. However, your low BMI of 16.4 indicates underweight status, which may pose risks for recovery and overall health. Balancing your high activity level with adequate caloric intake is crucial to prevent nutrient deficiencies.\n- Zone Minutes and Intensity Feedback: With 93 zone minutes per week, consider increasing this to enhance cardiovascular fitness. Aim for 120-150 zone minutes weekly by incorporating higher intensity intervals during your sessions, or extending the duration of your workouts by 10-15 minutes per session.\n- Strength/Cardio Tips: ['Incorporate strength training 2-3 times per week. Focus on compound movements like squats, deadlifts, and bench presses with 3 sets of 8-12 reps. Include bodyweight exercises such as push-ups, lunges, and planks for additional strength.', 'For cardio, aim for 30 minutes of moderate-intensity exercise (like jogging or cycling) 3 times a week, or include 20 minutes of high-intensity interval training (HIIT) once a week.']\n- Weekly Goals: ['Increase weekly steps to 65000.', 'Maintain 5 exercise sessions per week.', 'Raise zone minutes to 120.']\n\n**3) Sleep & Well-being**\n- Sleep Review: Your average sleep duration of 7.45 hours is within a healthy range, but given your high activity level and low BMI, it is crucial to ensure that your sleep quality is optimal for recovery. Late caffeine consumption or insufficient carbohydrate intake before bed could negatively impact your sleep quality and recovery. Additionally, low caloric intake may lead to energy deficits that can affect sleep patterns. Consider timing your meals and snacks appropriately to support your sleep and recovery.\n- Suggestions: ['Aim to have your last meal or snack at least 2-3 hours before bedtime to allow for digestion.', 'Incorporate complex carbohydrates (like quinoa or brown rice) in your evening meal to promote serotonin production, which aids in sleep.', \"Establish a calming pre-sleep routine, such as reading or meditating, to signal to your body that it's time to wind down.\", 'Limit caffeine intake in the afternoon and evening to prevent sleep disturbances.', 'Ensure adequate hydration throughout the day, but reduce fluid intake an hour before bed to minimize nighttime awakenings.']\n\n**4) Weekly Summary**\n- Summary: This week, focus on balancing your high activity level with adequate caloric intake to support your health and recovery. Increase your intake of healthy fats, protein sources, and calcium-rich alternatives while avoiding gluten. Additionally, aim to enhance your cardiovascular fitness and ensure optimal sleep quality through meal timing and a calming pre-sleep routine.\n- Goals: ['Increase weekly nut intake to 200 g.', 'Incorporate strength training 2-3 times this week.', 'Raise zone minutes to 120 by adding higher intensity intervals.', 'Include complex carbohydrates in your evening meal.', 'Aim for 65000 weekly steps.']"},

             ]

data_path = "/content/diet_finetune_examples.jsonl"
data = []
# with open(data_path, "r") as f:
#     for line in f:
#         item = json.loads(line)
#         user_msg = next(msg["content"] for msg in item["messages"] if msg["role"] == "user")
#         assistant_msg = next(msg["content"] for msg in item["messages"] if msg["role"] == "assistant")
#         prompt = f"<|user|>\n{user_msg}\n<|assistant|>\n{assistant_msg}"
#         data.append({"text": prompt})

data = [{"text": f"<|user|>\n{ex['input']}\n<|assistant|>\n{ex['output']}"} for ex in new_data]
# Convert to Hugging Face Dataset
dataset = Dataset.from_list(data)
dataset = dataset.train_test_split(test_size=0.1)

# Load tokenizer and base model
model_id = "ContactDoctor/Bio-Medical-Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# --- Use BitsAndBytesConfig for 4-bit loading ---
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config, # Use quantization_config instead of load_in_8bit
    device_map="auto",
    torch_dtype=torch.bfloat16, # Use bfloat16 for compute dtype
    low_cpu_mem_usage=True,
)

model = prepare_model_for_kbit_training(model)

# Apply LoRA PEFT configuration
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

# Tokenization function
def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=512)

tokenized_datasets = dataset.map(tokenize, batched=True)

# Training arguments
training_args = TrainingArguments(
    output_dir="./diet-finetuned-model",
    per_device_train_batch_size=1,  # Keep reduced batch size
    per_device_eval_batch_size=1,   # Keep reduced batch size
    num_train_epochs=3,
    logging_dir="./logs",
    save_total_limit=2,
    learning_rate=2e-4,
    fp16=False, # Set to False when using bfloat16 or 4-bit
    bf16=True, # Enable bfloat16 if supported by GPU
    save_strategy="epoch",
    report_to="none",
    gradient_checkpointing=True, # Keep gradient checkpointing enabled
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Start fine-tuning
trainer.train()

# Save final LoRA adapter
model.save_pretrained("./WellBeing_LLM")
tokenizer.save_pretrained("./WellBeing_LLM")

In [ ]:
adapter_path = "AnjaliNV/WellBeing_LLM"
model.push_to_hub(adapter_path, use_auth_token=True)
tokenizer.push_to_hub(adapter_path, use_auth_token=True)

#test

In [ ]:
from time import sleep
from typing_extensions import final
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
from langchain.schema import Document
import gc




# === Load FAISS vector store ===
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

vectorstore = FAISS.load_local(
    "/content/drive/MyDrive/rag_index",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)

foodstore = FAISS.load_local(
    "/content/drive/MyDrive/food_faiss_index",
    embeddings=embeddings,
    allow_dangerous_deserialization=True
)

# === Load LLaMA 3 model with LoRA adapter ===
base_model_id = "ContactDoctor/Bio-Medical-Llama-3-8B"
adapter_path = "AnjaliNV/WellBeing_LLM"

tokenizer = AutoTokenizer.from_pretrained(base_model_id)
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)
model = PeftModel.from_pretrained(base_model, adapter_path)
model.eval()

# === Define RAG-based response generation ===
def generate_step(prompt, temperature=0.3, max_tokens=256):
    try:
        print(f"Generating response with prompt length: {len(prompt)}")
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048).to(model.device)
        input_len = inputs["input_ids"].shape[1]

        print(f"Input token length: {input_len}")

        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                max_new_tokens=max_tokens,
                temperature=0.3,
                top_p=0.9,
                top_k=50,
                do_sample=True,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                repetition_penalty=1.05,
                use_cache=False,
            )

        generated_tokens = outputs[0][input_len:]
        response = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
        print(f"Generated response length: {len(response)}")
        return response
    except Exception as e:
        print(f"Error in generate_step: {e}")
        return f"Error generating response: {str(e)}"

def cleanup_gpu_memory():
    """Enhanced GPU memory cleanup"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
    gc.collect()

def generate_with_cot(user_profile, temperature=0.3, max_tokens=512, k=3):

    search1 = f"""
    Using the given user_profile:
    {user_profile}

    1) Find meal recommendations that match the user's dietary preference from user_profile.
    2) Respect their allergies, and include the recommended quantities per week.
    3) Quantities should be appropriate for their diet type, age, and other profile details.
    """

    # Retrieve relevant docs from vectorstore as context
    docs = vectorstore.similarity_search(search1, k=k)
    context = "\n\n".join(doc.page_content for doc in docs)

#     print("fetched context is ", context)

    prompt1 = f"""
      SYSTEM: You are a qualified well being recomender. Use explicit calculations, clear units, and do NOT invent facts not supported by the provided Context.

      CONTEXT (retrieved documents):
      {context}
      --- end of context ---

      Input:
      {user_profile}
      --- end of profile ---


          """

    final_response  = generate_step(prompt1, temperature, max_tokens)
#     food_response  = generate_step(prompt1, temperature, max_tokens)
#     cleanup_gpu_memory()

#     search2 = f"""
#     Given the following user profile:
#     {user_profile}

#     1) Identify the recommended zone minutes of exercise appropriate for the user's age group.
#     2) Suggest suitable strength training and cardiovascular exercises tailored to their profile.
#       """
#     docs = vectorstore.similarity_search(search2, k=k)
#     context2 = "\n\n".join(doc.page_content for doc in docs)
#     print("Physical context length:", len(context2))
#     prompt2 = f"""
#     SYSTEM: You are a qualified physical exercise recommender.

#     CONTEXT (retrieved documents):
#     {context2}
#     --- end of context ---

#     USER PROFILE:
#     {user_profile}
#     --- end of profile ---


#     TASK:- Using the Context and the user profile above, please provide a structured response for Physical Activity including these four parts:
#     1) Activity Assessment: Briefly evaluate the user’s current physical activity level based on their profile and any available info.
#     2) Zone Minutes and Intensity Feedback: Specify the recommended zone minutes (e.g., moderate, vigorous) suitable for their age group, including guidance on exercise intensity.
#     3) Strength/Cardio Tips: Suggest practical strength training and cardiovascular exercises that fit the user’s profile, preferences, and constraints.
#     4) Weekly Goals: Outline realistic weekly physical activity goals, including duration and frequency, to promote health and fitness.
#      """

#     physical_response  = generate_step(prompt2, temperature, max_tokens)
#     cleanup_gpu_memory()


#     search3 = f"""
#     Given the user profile:
#     {user_profile}

#     1) Determine the user’s current sleep duration and mood status.
#     2) Provide tailored recommendations to improve their sleep quality and mood, considering their age and sex.

#     """
#     docs = vectorstore.similarity_search(search3, k=k)
#     context3 = "\n\n".join(doc.page_content for doc in docs)
#     print("Sleep context length:", len(context3))
#     prompt3 = f"""
#     SYSTEM: You are a qualified well-being recommender.
#     CONTEXT (retrieved documents):
#     {context3}
#     --- end of context ---

#     USER PROFILE:
#     {user_profile}
#     --- end of profile ---

#     TASK: Using the provided context and user profile, deliver a structured response on Sleep and Mood covering these two parts:
#     1) Sleep and Mood Review: Analyze the user’s current sleep patterns and mood status based on the available information.
#     2) Suggestions: Offer actionable recommendations to improve the user’s sleep schedule and mood, if needed.
#     """
#     sleep_response  = generate_step(prompt3, temperature, max_tokens)
#     cleanup_gpu_memory()

#     final_response = f"""
#     Food Recommendation:
#     {food_response}
#     Physical Activity Recommendation:
#     {physical_response}
#     Sleep and Mood Recommendation:
#     {sleep_response}
#     """
#     print("final response is ", final_response)
    return final_response

# === Gradio UI ===
iface = gr.Interface(
    fn=generate_with_cot,
    inputs=[gr.Textbox(lines=10, label="Enter User Profile & Data"),
            gr.Slider(0.1, 1.0, value=0.3, label="Temperature"),
            gr.Slider(64, 1024, step=64, value=512, label="Max Tokens"),
            gr.Slider(1, 10, step=1, value=3, label="Top K Documents")],
    outputs=gr.Textbox(label="Health & Wellness Recommendation"),
    title="Chain-of-Thought RAG Wellness Advisor"
)

iface.launch(share=True, debug=True)

